In [1]:
import math
import numpy as np
import copy
import re
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from transformers import GPT2Tokenizer
from torchinfo import summary
from pathlib import Path
from datasets import load_dataset


tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

full_dataset = load_dataset("skeskinen/TinyStories-GPT4", split="train")
full_dataset = full_dataset.remove_columns([c for c in full_dataset.column_names if c not in ["story", "features"]])
assert len(full_dataset) == 2745100
splits = full_dataset.train_test_split(test_size=10000, seed=42, shuffle=True)
train_dataset = splits["train"]
test_dataset  = splits["test"]
assert len(train_dataset) == 2735100
assert len(test_dataset)  == 10000
assert train_dataset[0]["story"][:33] == "One day, a little girl named Lily"
assert train_dataset[0]["features"] == ["Dialogue", "Conflict"]

#Vocabulary Section

PAD_TOKEN = "[PAD]"
BOS_TOKEN = "[BOS]" #Beginning of Story
EOS_TOKEN = "[EOS]" #End of Story
TAG_1_TOKEN = "[TAG_1]" #BadEnding
TAG_2_TOKEN = "[TAG_2]" #Conflict
TAG_3_TOKEN = "[TAG_3]" #Dialogue
TAG_4_TOKEN = "[TAG_4]" #Foreshadowing
TAG_5_TOKEN = "[TAG_5]" #MoralValue
TAG_6_TOKEN = "[TAG_6]" #Twist
tags = {TAG_1_TOKEN: "BadEnding", TAG_2_TOKEN: "Conflict", TAG_3_TOKEN: "Dialogue", TAG_4_TOKEN: "Foreshadowing", TAG_5_TOKEN: "MoralValue", TAG_6_TOKEN: "Twist"}

#Define special tokens
special_tokens = {
    "pad_token": "[PAD]",
    "bos_token": "[BOS]",
    "eos_token": "[EOS]",
    "additional_special_tokens": ["[TAG_1]", "[TAG_2]", "[TAG_3]", "[TAG_4]", "[TAG_5]", "[TAG_6]"]
}

tokenizer.add_special_tokens(special_tokens) #Adding special tokens

vocab_size = len(tokenizer)
print("Vocab size:", vocab_size)

#PREPROCESSING SECTION

def get_tags_token(features): #From a list of tags it gets the list of corresponding token IDs
  tag_token_ids = []
  for tag in tags:
    if tags[tag] in features:
      tag_token_ids.append(tokenizer.encode(tag)[0])
  return tag_token_ids

def build_data(dataset, context_length): #Converts stories into tensors of l+1 token-sequences for later context-target splitting.
  stories = []
  for story in dataset:
    encoded = tokenizer.encode(story["story"])
    tag_vec = get_tags_token(story["features"])
    num_tag = len(tag_vec)
    L = context_length + 1 - num_tag
    # Create the full encoded sequence with BOS, tags, story, and EOS
    story_enc = [tokenizer.encode(PAD_TOKEN)[0]] * (context_length-1-num_tag) + [tokenizer.encode(BOS_TOKEN)[0]] + encoded + [tokenizer.encode(EOS_TOKEN)[0]]
    contexts = []
    for i in range(len(story_enc) - L + 1):
      context = tag_vec + story_enc[i:i+L]
      contexts.append(context)
    # Only process if there are any valid contexts to create
    if contexts:
        contexts_story = torch.tensor(contexts, dtype=torch.long)
        stories.append(contexts_story)
  return stories

l = 50 #Context lenght
random_seed = 42
dim_subset = 100000
fraction = (dim_subset + 1) / len(train_dataset)
subset = train_dataset.train_test_split(train_size = fraction, seed = random_seed)["train"]
dim_data = int(dim_subset*0.9) #split 90/10

data_train = build_data(subset.select(range(dim_data)), l) #building the training dataset
data_val = build_data(subset.select(range(dim_data, dim_subset)), l) #building the validation dataset

#MODEL ARCHITECTURE

emb_dim = 256
hidden_dim_1 = 768
hidden_dim_2 = 1024
num_heads = 8

encoder_layer_1= nn.TransformerEncoderLayer(
    d_model=emb_dim,
    nhead=num_heads,
    dim_feedforward=hidden_dim_1,
    batch_first=True,
)

encoder_layer_2= nn.TransformerEncoderLayer(
    d_model=emb_dim,
    nhead=num_heads,
    dim_feedforward=hidden_dim_2,
    batch_first=True,
)

class LearnedPositionalEmbedding(nn.Module): #to know in which point of the sequence is the token
    """
    Adds a trainable position vector to each token embedding.
    Expects input shape [B, L, D] (batch_first).
    """
    def __init__(self, max_length: int, dim: int):
        super().__init__()
        self.pos_emb = nn.Embedding(max_length, dim)

    #B = batch size, L = sequence lenght, D = embedding dimension

    def forward(self, x):                   # x: [B, L, D]
        L = x.size(1)
        pos = torch.arange(L, device=x.device)          # [L] generates a tensor the tensor of the position in the sequence: [0, 1, 2, ..., L-1]
        pos = pos.unsqueeze(0)                          # [1, L] adds the batch dimension
        return x + self.pos_emb(pos)                    # adds the info of the position to the embedding vector x

class SelectLastToken(nn.Module):
    """
    Keeps only the hidden state of the *final* token in the sequence.
    Input  : [B, L, D]
    Output : [B, D]
    """
    def forward(self, x):
        return x[:, -1, :]

padding_index = tokenizer.encode(PAD_TOKEN)[0]

model = nn.Sequential(
    nn.Embedding(vocab_size, emb_dim, padding_idx=padding_index),
    LearnedPositionalEmbedding(l, emb_dim),
    nn.TransformerEncoder(encoder_layer_1, num_layers=3),
    nn.Dropout(0.1),
    nn.TransformerEncoder(encoder_layer_2, num_layers=3),
    nn.Dropout(0.1),
    nn.TransformerEncoder(encoder_layer_2, num_layers=3),
    nn.Dropout(0.1),
    nn.TransformerEncoder(encoder_layer_1, num_layers=3),
    nn.Dropout(0.1),
    SelectLastToken(),
    nn.Linear(emb_dim, vocab_size)
)

summary(model, input_size=(emb_dim, l), dtypes=['torch.IntTensor'], device='cpu')

#TRAINING SECTION

def train(model, optimizer, data_train, data_val, epochs, device):
    model.to(device)
    # Data loaders
    train_loader = torch.utils.data.DataLoader(
        data_train, batch_size=1, shuffle=True,
    )
    val_loader = torch.utils.data.DataLoader(
        data_val, batch_size=1, shuffle=False
    )
    # Initial evaluation
    evaluate(model, val_loader, device)
    loss_fn = nn.CrossEntropyLoss()
    for epoch in range(epochs):
        model.train()
        train_losses = []
        # Training loop
        for context in train_loader:
            story = context[0].to(device)
            data = story[:, :-1].to(device)
            target = story[:, -1].to(device)
            optimizer.zero_grad()
            output = model(data)
            # Reshape to (batch*l, vocab_size) and (batch*l)
            output = output.reshape(-1, output.size(-1))
            target = target.reshape(-1)
            loss = loss_fn(output, target)
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())
        train_loss = float(np.mean(train_losses))
        print(f"Epoch {epoch+1}/{epochs} | Train Loss: {train_loss:.4f}")
        # Validation
        val_loss = evaluate(model, val_loader, device)


def evaluate(model, loader, device):
    model.to(device)
    model.eval()
    total_loss = 0.0
    loss_fn = nn.CrossEntropyLoss()
    with torch.no_grad():
        for context in loader:
            story = context[0].to(device)
            data = story[:, :-1].squeeze().to(device)
            target = story[:, -1].squeeze().to(device)
            output = model(data)
            output = output.reshape(-1, output.size(-1))
            target = target.reshape(-1)
            total_loss += loss_fn(output, target).item()
    average_loss = total_loss / len(loader)
    print(f"Validation Loss: {average_loss:.4f} | Perplexity: {math.exp(average_loss):.2f}")
    return average_loss


device = 'cuda' if torch.cuda.is_available() else 'cpu' # Detect device automatically
print(f"Using device: {device}")

num_epochs = 10
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
train(model, optimizer, data_train, data_val, epochs=num_epochs, device=device)

#TEXT GENERATING SECTION

def padding_truncate(seq, context_length):
  n = len(seq)
  copia = copy.copy(seq)
  if n > context_length:
    copia = copia[-context_length:]
  elif n < context_length:
    for i in range(context_length-n):
      copia.append(tokenizer.encode(PAD_TOKEN)[0])
  return copia


def top_p_filtering(logits, top_p = 0.9, filter_value = -float("inf")): #to implement top_p sampling
  sorted_logits, sorted_indices = torch.sort(logits, descending = True) #decrescent order
  cumulative_prob = torch.cumsum(torch.softmax(sorted_logits, dim = -1), dim = -1)
  sorted_indices_to_remove = cumulative_prob > top_p #create a boolean mask
  sorted_indices_to_remove[:, 1:] = sorted_indices_to_remove[:, :-1].clone()
  sorted_indices_to_remove[:, 0] = False
  indices_to_remove = sorted_indices[sorted_indices_to_remove]
  logits[-1][indices_to_remove] = filter_value
  return logits


def generate_text(model, tags = [], device = None, top_p = 0.9, context_length = l):
  if device is None:
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
  model.eval()
  for tag in tags:
    if tag not in ["BadEnding", "Conflict", "Dialogue", "Foreshadowing", "MoralValue", "Twist"]:
      return str("This tag: " + tag + " is invalid. Choose tag among: BadEnding, Conflict, Dialogue, Foreshadowing, MoralValue, Twist")
  tag_vec = get_tags_token(tags)
  num_tags = len(tag_vec)
  enc = tokenizer.encode(PAD_TOKEN)*(context_length-1-num_tags) + tokenizer.encode(BOS_TOKEN) #create input for the model
  eos_token_id = tokenizer.encode(EOS_TOKEN)[0]
  while enc[-1] != eos_token_id and len(enc)<350:
    x = padding_truncate(enc, context_length - num_tags)
    x = tag_vec + x
    x = torch.tensor(x).unsqueeze(0).to(device)
    with torch.no_grad():
      output = model(x)
      #add a bias to EOS token that increases with the lenght of the story from 0 to 5
      #starts with 0 at 200 tokens, reaching 5 at 250 tokens
      len_story = len(enc) - context_length + num_tags
      if len_story > 200:
        eos_bias = min(5, 0.1 * (len_story - 200))
        output[-1][eos_token_id] += eos_bias
      logits = top_p_filtering(output, top_p=top_p) #top_p
      next_word = torch.multinomial(torch.softmax(logits, -1), 1).item() #predict the next token
      enc.append(next_word)
  # Decode from the token after the BOS token and before the EOS token (if present)
  story = enc[context_length - num_tags : -1 if enc[-1] == eos_token_id else None]
  text = tokenizer.decode(story)
  print("Generating story with", tags)
  print(text) #print instead of return so that we don't see anymore characters like \n

#TEST SECTION

def avg_cross_entropy(token_probs):
    """
    Average over all stories of the sum of -log p for every ground-truth token.

    token_probs : list[list[float]]
        Probabilities assigned to the correct token at each position.
    """
    return np.average([-np.log(story_token_probs).sum() for story_token_probs in token_probs])

#Building the test_dataset to have a clear structure
test_subset = test_dataset.select(range(100))  # Select first 100 test examples
test_data = [
    {"story": s, "features": f}
    for s, f in zip(test_subset["story"], test_subset["features"])
]

#Test the model on the test_dataset with the function provided

model.eval()

token_probs = []
for story in test_data:
    tag_vec = get_tags_token(story["features"])
    num_tags = len(tag_vec)
    true_tokens = tokenizer.encode(story["story"]) + tokenizer.encode(EOS_TOKEN)
    input = tokenizer.encode(PAD_TOKEN)*(l-1-num_tags) + tokenizer.encode(BOS_TOKEN)
    probs = []
    #for each true_token we take the probability generated by the model
    for next_token in true_tokens:
        x = padding_truncate(input, l - num_tags)
        x = tag_vec + x
        x = torch.tensor(x).unsqueeze(0).to(device)
        with torch.no_grad():
            output = model(x) #predicts the probability of the next_token based on the last l tokens
            distrib = torch.softmax(output, -1)
            prob = distrib[-1][next_token].item()
            probs.append(prob)
        input.append(next_token)
    token_probs.append(probs)

print(f"Average cross-entropy {avg_cross_entropy(token_probs):.2f} nats/story")

KeyboardInterrupt: 